In following the

In [1]:
import pandas as pd
from queries import *

In [2]:
#load the ontology from "./ontologies/" and perform reasoning
onto_name= "afo_dataset_1-m" #input name of the ontology

#perform reasoning, get dois from publications in the extended ontology
onto_pub_list,onto,df=reasoning_dois_onto(onto_name)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
 
display(df)


* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/name belongs to more than one entity types: [owl.AnnotationProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp C:\Users\chern\anaconda3\envs\catnew\lib\site-packages\owlready2\hermit;C:\Users\chern\anaconda3\envs\catnew\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/chern/AppData/Local/Temp/tmpyx7fnxaa
* Owlready2 * HermiT took 95.36029005050659 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


The afo_dataset_1-m ontology consists of following publications:


,DOI,Title
0,10.1016/0304-5102(93)87113-m,Vapour phase propene hydroformylation catalyzed by the Rh/Al system on silica
1,10.1021/acscatal.0c04684.s001,Promotion of Inorganic Phosphorus on Rh Catalysts in Styrene Hydroformylation: Geometric and Electronic Effects
2,10.1021/acscatal.1c00705.s002,Theoretical Study of Ethylene Hydroformylation on Atomically Dispersed Rh/Al2O3 Catalysts: Reaction Mechanism and Influence of the ReOx Promoter
3,10.1021/acscatal.1c02014.s001,Activity Promotion of Rh8xCoxP4 Bimetallic Phosphides in Styrene Hydroformylation: Dual Influence of Adsorption and Surface Reaction
4,10.1021/jacs.1c09665.s001,Intermetallic Nanocatalyst for Highly Active Heterogeneous Hydroformylation
5,10.1016/j.apcata.2018.02.019,Rhodium incorporated monometallic cobalt hydrotalcite-type materials: Preparation and its applications for the hydroformylation of alkenes
6,10.1021/jacs.2c11075.s001,Ethene Hydroformylation Catalyzed by Rhodium Dispersed with Zinc or Cobalt in Silanol Nests of Dealuminated Zeolite Beta
7,10.1016/s0920-5861(00)00261-3,Ethylene hydroformylation and carbon monoxide hydrogenation over modified and unmodified silica supported rhodium catalysts
8,10.1016/j.apcata.2013.10.019,Experimental investigation of ethylene hydroformylation to propanal on Rh and Co based catalysts
9,10.1016/s1381-1169(97)00035-6,Formation of bimetallic RhCo3 clusters from monometallic carbonyl clusters on SiO2 as probed by hydroformylation


define doi if only publications similar to specific publication in ontology should be queried from scopus\
if all publications in ontology should be used for query define the variable as:\
`doi = None`

In [7]:
doi = r'10.1021/acs.iecr.9b03598.s001' 

Retrieve an abstract of the specific publication in the ontology given the DOI of the publication 

In [13]:
doi_1=r'10.1016/1381-1169(96)00243-9' 
abstract=get_abstr(doi_1)

Abstract:The reduction of cobalt and rhodium salts coadsorbed on silica by aqueous NaBH4 at 273 K in Ar allows the synthesis of catalytic systems formed by very small rhodium crystallites (< 4 nm) and cobalt oxide/hydroxide. The presence of an unreduced cobalt species is well documented by TPR and XPS. The cobalt oxide is probably deposited on the rhodium surface, obscuring a large amount of the active metal centers. As can be judged by FT-IR and XRD data the morphology of the system is not modified by thermal treatments in CO and H2. The system resulted inactive for the atmospheric hydroformylation of propene, but actively catalyzed the reaction when a slight pressure (506 kPa) was applied. The high values of chemoselectivity towards hydroformylation (R = 0.75) and regioselectivity to linear aldehydes (S(L) = 96) can be due to the electronic and steric effects of the cobalt oxide layer.


Following cells represent possible queries for reactions

- get list of all reactions for corresponding doi or from all publications if doi=None

In [4]:
list_reac_doi,_ = get_reaction(reac=None,doi=doi) 
reac_all = [*set([i[0].lower() for i in list_reac_doi])]
print(reac_all)

['hydrogenation', 'hydroformylation']


- get the list of all publications in ontology where same reactions as in input publication were mentioned

In [10]:
list_reac_doi,_ = get_reaction(reac = None,doi = doi) #get list of all reactions mentioned in given doi (doi should be part 
                                                    #of ontology)
same_reac_doi = []
for i in list_reac_doi:
    reac_doi = get_reaction(reac = i[0],doi = None)
    for c in reac_doi:
        if c not in same_reac_doi and c[0] != doi:
            same_reac_doi.append(c)     #output example: [['10.1016/0304-5102(93)87113-m'],['10.1016/1381-1169(96)00243-9']]

print(list_reac_doi)
print(same_reac_doi)

[['hydroformylation'], ['hydrogenation']]
[['10.1016/0304-5102(93)87113-m'], ['10.1021/acscatal.1c00705.s002'], ['10.1021/acscatal.1c02014.s001'], ['10.1021/acscatal.1c04359'], ['10.1021/acscatal.7b00499.s001'], ['10.1021/acscatal.9b02111.s001'], ['10.1021/jacs.1c09665.s001'], ['10.1016/j.apcata.2018.02.019'], ['10.1021/jacs.2c11075.s001'], ['10.1016/1381-1169(96)00243-9'], ['10.1016/s0920-5861(00)00261-3'], ['10.1016/j.apcata.2013.10.019'], ['10.1016/s1381-1169(97)00035-6'], ['10.1021/acs.iecr.0c03437.s001'], ['10.1021/acs.iecr.9b03598.s001'], ['10.1021/acsami.0c21749.s001']]


- get the list of publications, that mention the specific reaction

In [12]:
reac="hydrogenation"
list_reac_doi,_ = get_reaction(reac=reac,doi=None)  #output example: [['10.1016/0304-5102(93)87113-m'],['10.1016/1381-1169(96)00243-9']]
print(list_reac_doi)

[['10.1021/acscatal.0c04684.s001'], ['10.1021/acscatal.1c00705.s002'], ['10.1021/acscatal.1c04359'], ['10.1021/jacs.2c11075.s001'], ['10.1016/s0920-5861(00)00261-3'], ['10.1016/j.apcata.2013.10.019']]


- get list of all publications which mention the specific reaction and reactions which have this specific reaction as part of their label

In [3]:
reac="hydroformylation" #specify a reaction

list_reac_doi, df_reac = get_reaction(reac="hydroformylation",doi=None, include_all=True)

display(df_reac)

,DOI,Title
0,10.1016/0304-5102(93)87113-m,hydroformylation
1,10.1021/acscatal.0c04684.s001,hydroformylation
2,10.1021/acscatal.1c00705.s002,hydroformylation
3,10.1021/acscatal.1c02014.s001,hydroformylation
4,10.1021/acscatal.1c04359,hydroformylation
5,10.1021/acscatal.7b00499.s001,hydroformylation
6,10.1021/acscatal.9b02111.s001,hydroformylation
7,10.1021/jacs.1c09665.s001,hydroformylation
8,10.1016/j.apcata.2018.02.019,hydroformylation
9,10.1021/jacs.2c11075.s001,hydroformylation


- get list of all entities mentioned in specific publication

In [ ]:
list_ent=get_entList(list_type='all',entity= None,doi=doi)
print([i[1] for i in list_ent])

With following code the retrieval if all reactants, products, support materials, catalysts is performed, which are mentioned in one specific publication or from all publication in extended ontology (if doi=None) 
with `get_synonyms` all possible synonyms of chemical entities will be retrieved.\
For example from "rhodium atom " class, synonyms such as "Rh", "rhodium", "45Rh" are retrieved

In [18]:
sup_list=get_support(sup=None,doi=doi)
_,sup_all=get_synonyms(sup_list)
print('all queried synonyms of support:')
print(list(sup_all.values()))
cat_list=get_catalyst(cat=None,doi=doi,include_all=True)
_,cat_all=get_synonyms(cat_list)
print('\nall queried synonyms of catalytic components:')
print(list(cat_all.values()))
reactant_list=get_entList('reactant',entity= None,doi=doi)
reactant_all,_=get_synonyms(reactant_list)
print('\nall queried synonyms of reactants:')
print(reactant_all)
product_list=get_entList('product',entity= None,doi=doi)
product_all,_=get_synonyms(product_list)
print('\nall queried synonyms of products:')
print(product_all)
cat_list_full=get_catalyst_full(doi = doi)
_,cat_full_all=get_synonyms(cat_list_full)
print('\nall queried catalyst entities:')
print(list(cat_full_all.values()))

all queried synonyms of support:
[['S-1'], ['S-1-II'], ['silicalite-1']]

all queried synonyms of catalytic components:
[['rodio', 'rhodium', '45Rh', 'Rh', 'Rh(111)', 'rhodium atom'], ['Silica, amorphous', 'silicalite', '[SiO2]', 'SiO2', 'silicon(IV) oxide', 'silica', 'Siliziumdioxid', 'silicic anhydride', '(SiO2)n', 'Kieselsaeureanhydrid', 'silicon dioxide'], ['supported Rh2O3/S-1 catalyst', 'supported Rh2O3 supported on S-1 catalyst'], ['Rh2O3@S-1-II', 'Rh2O3 supported on S-1-II'], ['Rh2O3@S-1'], ['zeolite'], ['S-1'], ['Rh2O3', 'oxo(oxorhodiooxy)rhodium'], ['rhodium oxide', 'oxorhodium']]

all queried synonyms of reactants:
['1-hexene', 'hex-1-ene', 'adacene 12', 'alpha-dodecene', '1-dodecene', 'dodec-1-ene', 'neodene 12', 'alpha-dodecylene', 'dodecene-1', 'n-dodec-1-ene']

all queried synonyms of products:
[]

all queried catalyst entities:
[['supported Rh2O3/S-1 catalyst', 'supported Rh2O3 supported on S-1 catalyst'], ['Rh2O3@S-1-II', 'Rh2O3 supported on S-1-II']]


Perform a search in scopus using collected information from ontology\
Queries have following structure: 'TITLE-ABS-KEY("reaction"AND"catalyst"AND"support material"AND"reactant"AND"product")' in all possible combinations

In [3]:
doi=None
df_all=scopus_seach_process(doi, onto_pub_list)

Scopus429Error: Quota Exceeded

All distinct results are saved in sheet 'distinct_eid'.\
Filter found publication according to the start and end date\
If no filtering nessesary define `start_date` as follows:\
`get_synonyms = None`

In [4]:
start_date = '2019-01-01'
end_date = '2023-12-31' 

distinct_df, filtered_df, filtered_df_doi, filtered_df_doi_new = filter_date(start_date,end_date, df_all,onto_pub_list)

NameError: name 'df_all' is not defined

save from scopus quieried publication information in Excel

In [113]:
output_path='./output/output.xlsx' #define the output path

save_in_excel(output_path, df_all, distinct_df, filtered_df, filtered_df_doi, filtered_df_doi_new )